# Spotify MRS - Model Development

## Table of Contents
 * [1.Data Import](#chapter1)
 * [2.Content Baesd Filtering](#chapter2)
     * [2.1.Choose the Playlist](#section_2_1)
     * [2.2.Extract the Features](#section_2_2)
     * [2.3.Find the Similaraties](#section_2_3)
     * [2.4.Recommendation](#section_2_4)

## 1.Data Import

In [126]:
# Load the final feature set got from feature engineering
import pandas as pd
features=pd.read_csv('final_features.csv')
# Get the summary for features
features.head()

,id,genre|_hip_hop,genre|a_cappella,genre|abstract_beats,genre|abstract_hip_hop,genre|acoustic_blues,genre|acoustic_pop,genre|acoustic_rock,genre|adult_standards,genre|aesthetic_rap,...,valence,track_pop,subject|high,subject|low,subject|medium,polar|Negative,polar|Neutral,polar|Positive,mode|0,mode|1
0,0UaMYEvWZi0ZqiDOoHU3YI,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.824847,0.788235,0,1,0,0,1,0,1,0
1,0UaMYEvWZi0ZqiDOoHU3YI,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.824847,0.788235,0,1,0,0,1,0,1,0
2,0UaMYEvWZi0ZqiDOoHU3YI,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.824847,0.788235,0,1,0,0,1,0,1,0
3,0UaMYEvWZi0ZqiDOoHU3YI,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.824847,0.788235,0,1,0,0,1,0,1,0
4,0UaMYEvWZi0ZqiDOoHU3YI,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.824847,0.788235,0,1,0,0,1,0,1,0


In [127]:
# Load the challenge data
import json
challenge=json.load(open('Json_Data_set.json'))
fields=['pid','name','num_tracks', 'num_samples','num_holdouts']
challenge=pd.json_normalize(challenge['playlists'],'tracks',fields,errors='ignore')
# Get the summary of the challenge data
challenge.head()

,pos,artist_name,track_uri,artist_uri,track_name,album_uri,duration_ms,album_name,pid,name,num_tracks,num_samples,num_holdouts
0,0,AronChupa,spotify:track:66U0ASk1VHZsqIkpMjKX3B,spotify:artist:5vCOdeiQt9LyzdI87kt5Sh,Little Swing,spotify:album:4S5MLjwRSi0NJ5nikflYnZ,163809,Little Swing,1000000,Party,75,5,70
1,1,AronChupa,spotify:track:5MhsZlmKJG6X5kTHkdwC4B,spotify:artist:5vCOdeiQt9LyzdI87kt5Sh,I'm an Albatraoz,spotify:album:1qHVYbxQ6IS8YRviorKDJI,166848,I'm an Albatraoz,1000000,Party,75,5,70
2,2,Lorde,spotify:track:0GZoB8h0kqXn7XFm4Sj06k,spotify:artist:163tK9Wjr9P9DmM0AVK7lm,Yellow Flicker Beat - From The Hunger Games: M...,spotify:album:4UEPxQx0cTcYNsE0n32MHV,232506,Yellow Flicker Beat,1000000,Party,75,5,70
3,3,Lorde,spotify:track:35kahykNu00FPysz3C2euR,spotify:artist:163tK9Wjr9P9DmM0AVK7lm,White Teeth Teens,spotify:album:0rmhjUgoVa17LZuS8xWQ3v,216600,Pure Heroine,1000000,Party,75,5,70
4,4,Lorde,spotify:track:3G6hD9B2ZHOsgf4WfNu7X1,spotify:artist:163tK9Wjr9P9DmM0AVK7lm,Team,spotify:album:0rmhjUgoVa17LZuS8xWQ3v,193058,Pure Heroine,1000000,Party,75,5,70


In [128]:
# Extract the track ID from track_uri
import re
challenge['track_uri']=challenge['track_uri'].apply(lambda x : re.findall(r'\w+$',x)[0])
challenge.rename(columns={'track_uri':'id'},inplace=True)
challenge.head()

,pos,artist_name,id,artist_uri,track_name,album_uri,duration_ms,album_name,pid,name,num_tracks,num_samples,num_holdouts
0,0,AronChupa,66U0ASk1VHZsqIkpMjKX3B,spotify:artist:5vCOdeiQt9LyzdI87kt5Sh,Little Swing,spotify:album:4S5MLjwRSi0NJ5nikflYnZ,163809,Little Swing,1000000,Party,75,5,70
1,1,AronChupa,5MhsZlmKJG6X5kTHkdwC4B,spotify:artist:5vCOdeiQt9LyzdI87kt5Sh,I'm an Albatraoz,spotify:album:1qHVYbxQ6IS8YRviorKDJI,166848,I'm an Albatraoz,1000000,Party,75,5,70
2,2,Lorde,0GZoB8h0kqXn7XFm4Sj06k,spotify:artist:163tK9Wjr9P9DmM0AVK7lm,Yellow Flicker Beat - From The Hunger Games: M...,spotify:album:4UEPxQx0cTcYNsE0n32MHV,232506,Yellow Flicker Beat,1000000,Party,75,5,70
3,3,Lorde,35kahykNu00FPysz3C2euR,spotify:artist:163tK9Wjr9P9DmM0AVK7lm,White Teeth Teens,spotify:album:0rmhjUgoVa17LZuS8xWQ3v,216600,Pure Heroine,1000000,Party,75,5,70
4,4,Lorde,3G6hD9B2ZHOsgf4WfNu7X1,spotify:artist:163tK9Wjr9P9DmM0AVK7lm,Team,spotify:album:0rmhjUgoVa17LZuS8xWQ3v,193058,Pure Heroine,1000000,Party,75,5,70


## 2.Content based Filtering

There are three parts in this section <br>
<br>
1.Choose the playlist from challenge data <br>
2.Extraxt the features <br>
3.Find the cosine similaraties

## 2.1.Choose the Playlist

In [129]:
# Choose the the playlist which pid=1000000
playlist=challenge[challenge.pid==1000000]
playlist

,pos,artist_name,id,artist_uri,track_name,album_uri,duration_ms,album_name,pid,name,num_tracks,num_samples,num_holdouts
0,0,AronChupa,66U0ASk1VHZsqIkpMjKX3B,spotify:artist:5vCOdeiQt9LyzdI87kt5Sh,Little Swing,spotify:album:4S5MLjwRSi0NJ5nikflYnZ,163809,Little Swing,1000000,Party,75,5,70
1,1,AronChupa,5MhsZlmKJG6X5kTHkdwC4B,spotify:artist:5vCOdeiQt9LyzdI87kt5Sh,I'm an Albatraoz,spotify:album:1qHVYbxQ6IS8YRviorKDJI,166848,I'm an Albatraoz,1000000,Party,75,5,70
2,2,Lorde,0GZoB8h0kqXn7XFm4Sj06k,spotify:artist:163tK9Wjr9P9DmM0AVK7lm,Yellow Flicker Beat - From The Hunger Games: M...,spotify:album:4UEPxQx0cTcYNsE0n32MHV,232506,Yellow Flicker Beat,1000000,Party,75,5,70
3,3,Lorde,35kahykNu00FPysz3C2euR,spotify:artist:163tK9Wjr9P9DmM0AVK7lm,White Teeth Teens,spotify:album:0rmhjUgoVa17LZuS8xWQ3v,216600,Pure Heroine,1000000,Party,75,5,70
4,4,Lorde,3G6hD9B2ZHOsgf4WfNu7X1,spotify:artist:163tK9Wjr9P9DmM0AVK7lm,Team,spotify:album:0rmhjUgoVa17LZuS8xWQ3v,193058,Pure Heroine,1000000,Party,75,5,70


## 2.2.Extract the Features

In [130]:
# Find all playlist song features
features_playlist = features[features['id'].isin(challenge['id'].values)]
features_playlist_final = features_playlist.drop(columns = "id")
features_playlist_vector=features_playlist_final.sum(axis=0)
features_playlist_vector

genre|_hip_hop               45.829672
genre|a_cappella             12.000000
genre|abstract_beats          0.000000
genre|abstract_hip_hop        2.147877
genre|acoustic_blues          0.000000
                              ...     
polar|Negative             5052.000000
polar|Neutral             45636.000000
polar|Positive             8944.000000
mode|0                    21650.000000
mode|1                    37982.000000
Length: 712, dtype: float64

In [131]:
# Find all non-playlist song features
features_nonplaylist = features[~features['id'].isin(challenge['id'].values)]
features_nonplaylist

,id,genre|_hip_hop,genre|a_cappella,genre|abstract_beats,genre|abstract_hip_hop,genre|acoustic_blues,genre|acoustic_pop,genre|acoustic_rock,genre|adult_standards,genre|aesthetic_rap,...,valence,track_pop,subject|high,subject|low,subject|medium,polar|Negative,polar|Neutral,polar|Positive,mode|0,mode|1
3306,1X5WGCrUMuwRFuYU1eAo2I,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.524440,0.235294,0,1,0,0,1,0,0,1
3307,1X5WGCrUMuwRFuYU1eAo2I,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.524440,0.235294,0,1,0,0,1,0,0,1
3308,1X5WGCrUMuwRFuYU1eAo2I,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.524440,0.235294,0,1,0,0,1,0,0,1
3309,1X5WGCrUMuwRFuYU1eAo2I,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.524440,0.235294,0,1,0,0,1,0,0,1
3558,1MYYt7h6amcrauCOoso3Gx,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.135438,0.000000,0,1,0,0,1,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59607,7ceisg9MpCeJoLaoPfyu0V,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.547862,0.447059,0,1,0,0,1,0,0,1
59610,23Y91ZN3eM2ABYDyYuD0X2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.505092,0.105882,0,1,0,0,1,0,1,0
59611,23Y91ZN3eM2ABYDyYuD0X2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.505092,0.105882,0,1,0,0,1,0,1,0
60808,1V2D3l085S3fQsVR7aYgQx,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.494908,0.341176,0,1,0,0,1,0,1,0


## 2.3.Find the Similaraties

In [132]:
# Load the raw data that contains track details
song=pd.read_csv('raw_data.csv')
song['track_uri']=song['track_uri'].apply(lambda x : re.findall(r'\w+$',x)[0])
song.rename(columns={'track_uri':'id'},inplace=True)
song.head()

,pid,name,description,modified_at,num_artists,num_albums,num_tracks,num_followers,num_edits,playlist_duration_ms,collaborative,pos,artist_name,id,artist_uri,track_name,album_uri,duration_ms,album_name
0,0,Throwbacks,NaN,1493424000,37,47,52,1,6,11532414,False,0,Missy Elliott,0UaMYEvWZi0ZqiDOoHU3YI,spotify:artist:2wIVse2owClT7go1WT98tk,Lose Control (feat. Ciara & Fat Man Scoop),spotify:album:6vV5UrXcfyQD1wu4Qo2I9K,226863,The Cookbook
1,0,Throwbacks,NaN,1493424000,37,47,52,1,6,11532414,False,1,Britney Spears,6I9VzXrHxO9rA9A5euc8Ak,spotify:artist:26dSoYclwsYLMAKD3tpOr4,Toxic,spotify:album:0z7pVBGOD7HCIB7S8eLkLI,198800,In The Zone
2,0,Throwbacks,NaN,1493424000,37,47,52,1,6,11532414,False,2,Beyoncé,0WqIKmW4BTrj3eJFmnCKMv,spotify:artist:6vWDO969PvNqNYHIOW5v0m,Crazy In Love,spotify:album:25hVFAxTlDvXbx2X2QkUkE,235933,Dangerously In Love (Alben für die Ewigkeit)
3,0,Throwbacks,NaN,1493424000,37,47,52,1,6,11532414,False,3,Justin Timberlake,1AWQoqb9bSvzTjaLralEkT,spotify:artist:31TPClRtHm23RisEBtV3X7,Rock Your Body,spotify:album:6QPkyl04rXwTGlGlcYaRoW,267266,Justified
4,0,Throwbacks,NaN,1493424000,37,47,52,1,6,11532414,False,4,Shaggy,1lzr43nnXAijIGYnCT8M8H,spotify:artist:5EvFsr3kj42KNv97ZEnqij,It Wasn't Me,spotify:album:6NmFmPX56pcLBOFMhIiKvF,227600,Hot Shot


In [147]:
# Find the cosine similaraties
from sklearn.metrics.pairwise import cosine_similarity
song_non_playlist = song[song['id'].isin(features_nonplaylist['id'].values)]
data={'cos_sim':cosine_similarity(features_nonplaylist.drop('id', axis = 1).values,
                                          features_playlist_vector.values.reshape(1, -1))[:,0],
      'id':features_nonplaylist['id']
     }

In [148]:
# Create the data frame for cosine similaraties
similaraties_df=pd.DataFrame(data)
similaraties_df

,cos_sim,id
3306,0.802245,1X5WGCrUMuwRFuYU1eAo2I
3307,0.802245,1X5WGCrUMuwRFuYU1eAo2I
3308,0.802245,1X5WGCrUMuwRFuYU1eAo2I
3309,0.802245,1X5WGCrUMuwRFuYU1eAo2I
3558,0.632903,1MYYt7h6amcrauCOoso3Gx
...,...,...
59607,0.825136,7ceisg9MpCeJoLaoPfyu0V
59610,0.727714,23Y91ZN3eM2ABYDyYuD0X2
59611,0.727714,23Y91ZN3eM2ABYDyYuD0X2
60808,0.711192,1V2D3l085S3fQsVR7aYgQx


## 2.4.Recommendation

In [153]:
# Murge the data
reco=pd.merge(similaraties_df,song_non_playlist)
# Sorting the data frame based on cosine similaraties
reco=reco.sort_values('cos_sim',ascending = False)
# Recommended 10 track and artist name for playlist of pid=10000
reco_track_artist=reco[['track_name','artist_name']]
reco_track_artist.drop_duplicates().head(10)

,track_name,artist_name
241,Call You Bae,JIMIN (AOA)
2553,Supersoaka,Kid Ink
812,Just The Same,Bruno Major
386,CONTACTS,BROCKHAMPTON
398,Miss You,bo en
989,In the morning - Version,Natural Self
402,Money Won't Pay,bo en
1662,Lost in the Supermarket,The Clash
1605,Conversation Piece,Kings of Leon
2731,Careers In Combat,Parquet Courts
